In [164]:
import numpy as np
import pandas as pd

# Data retrieved from 'Huge Stock Market Dataset'
# https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs

# The variable to be predicted will be the next day's maximum

In [165]:
# Read data

SPY = pd.read_csv('spy.us.txt')
SCHD = pd.read_csv('schd.us.txt')
VUG = pd.read_csv('vug.us.txt')

SPY = SPY.drop(columns = 'OpenInt')
SCHD = SCHD.drop(columns = 'OpenInt')
VUG = VUG.drop(columns = 'OpenInt')

In [166]:
# RSI is the 'relative strength index'
def rsi(values):
    up = values[values>0].mean()
    down = -1*values[values<0].mean()
    return 100 * up / (up + down)

# Adding previous day change
SPY['Momentum1D'] = (SPY['Close']-SPY['Close'].shift(1)).fillna(0)
SCHD['Momentum1D'] = (SCHD['Close']-SCHD['Close'].shift(1)).fillna(0)
VUG['Momentum1D'] = (VUG['Close']-VUG['Close'].shift(1)).fillna(0)

# Adding 14-day RSI to dataframes
SPY['RSI14D'] = SPY['Momentum1D'].rolling(center=False, window=14).apply(rsi).fillna(0)
SCHD['RSI14D'] = SCHD['Momentum1D'].rolling(center=False, window=14).apply(rsi).fillna(0)
VUG['RSI14D'] = VUG['Momentum1D'].rolling(center=False, window=14).apply(rsi).fillna(0)

In [167]:
# Creating Bollinger Bands and adding them to dataframes

def bbands(price, length, z):
    mu = price.rolling(window = length, center = False).mean()
    sigma = price.rolling(window = length, center = False).std()
    upperband = mu + (sigma*z)
    lowerband = mu - (sigma*z)
    return mu, upperband, lowerband

SPY['MiddleBB'], SPY['UpperBB'], SPY['LowerBB'] = bbands(SPY['Close'], length=20, z=1)
SCHD['MiddleBB'], SCHD['UpperBB'], SCHD['LowerBB'] = bbands(SCHD['Close'], length=20, z=1)
VUG['MiddleBB'], VUG['UpperBB'], VUG['LowerBB'] = bbands(VUG['Close'], length=20, z=1)

SPY = SPY.fillna(0)
SCHD = SCHD.fillna(0)
VUG = VUG.fillna(0)

In [168]:
# Finding the 1 day price/volume trend

SPY['PVT'] = (SPY['Momentum1D'] / SPY['Close'].shift(1)) * SPY['Volume']
SPY['PVT'] = SPY['PVT'] - SPY['PVT'].shift(1)
SPY = SPY.fillna(0)

SCHD['PVT'] = (SCHD['Momentum1D'] / SCHD['Close'].shift(1)) * SCHD['Volume']
SCHD['PVT'] = SCHD['PVT'] - SCHD['PVT'].shift(1)
SCHD = SCHD.fillna(0)

VUG['PVT'] = (VUG['Momentum1D'] / VUG['Close'].shift(1)) * VUG['Volume']
VUG['PVT'] = VUG['PVT'] - VUG['PVT'].shift(1)
VUG = VUG.fillna(0)

In [169]:
# Calculating the one-week price rate of change

SPY['PROC'] = 100*(SPY['Close'] - SPY['Close'].shift(5)) / SPY['Close'].shift(5)
SCHD['PROC'] = 100*(SCHD['Close'] - SCHD['Close'].shift(5)) / SCHD['Close'].shift(5)
VUG['PROC'] = 100*(VUG['Close'] - VUG['Close'].shift(5)) / VUG['Close'].shift(5)

SPY = SPY.fillna(0)
SCHD = SCHD.fillna(0)
VUG = VUG.fillna(0)

In [170]:
# Money flow index

def mfi(df):
    df['Typical Price'] = (df['High']+df['Low']+df['Close'])/3
    df['+ MF'] = np.where(df['Typical Price'] > df['Typical Price'].shift(1), df['Typical Price'], 0)
    df['- MF'] = np.where(df['Typical Price'] < df['Typical Price'].shift(1), df['Typical Price'], 0)
    df['MF Ratio'] = df['+ MF'].rolling(window=14,center=False).sum()/df['- MF'].rolling(window=14,center=False).sum()
    df['MoneyFlowIndex'] = 100 - 100 / (1 + df['MF Ratio'])
    
mfi(SPY)
mfi(SCHD)
mfi(VUG)

SPY = SPY.fillna(0)
SCHD = SCHD.fillna(0)
VUG = VUG.fillna(0)

SPY = SPY.drop(labels = ['Typical Price','+ MF','- MF','MF Ratio'], axis = 1)
SCHD = SCHD.drop(labels = ['Typical Price','+ MF','- MF','MF Ratio'], axis = 1)
VUG = VUG.drop(labels = ['Typical Price','+ MF','- MF','MF Ratio'], axis = 1)

In [171]:
# Adding the next day's high - the variable to be predicted by the neural net

SPY['NextHigh'] = SPY['High'].shift(-1)
SCHD['NextHigh'] = SCHD['High'].shift(-1)
VUG['NextHigh'] = VUG['High'].shift(-1)

SPY = SPY.fillna(0)
SCHD = SCHD.fillna(0)
VUG = VUG.fillna(0)

In [172]:
# Dropping rows without technical analysis indicators
# The first 20 rows have no bollinger bands

todrop = []
for i in range(19):
    todrop += [i]
    
SPY = SPY.drop(labels = todrop, axis = 0)
SCHD = SCHD.drop(labels = todrop, axis = 0)
VUG = VUG.drop(labels = todrop, axis = 0)

In [173]:
# Dropping final row since it has no prediction variable

SPY.drop(SPY.tail(1).index, inplace = True)
SCHD.drop(SCHD.tail(1).index, inplace = True)
VUG.drop(VUG.tail(1).index, inplace = True)

In [174]:
# Renumbering rows

SPY.reset_index(inplace = True)
SCHD.reset_index(inplace = True)
VUG.reset_index(inplace = True)

In [175]:
# Exporting dataframes as CSV files for use in other notebooks

SPY.to_csv('SPY_Technical_Analysis.csv')
SCHD.to_csv('SCHD_Technical_Analysis.csv')
VUG.to_csv('VUG_Technical_Analysis.csv')